In [1]:
import os
os.chdir("W:/RM/src")

In [2]:
from mmdetection_rm.work.work_resource import WorkResourceFactory
factory = WorkResourceFactory()
db = factory.resource_db

***violating paths***


In [3]:
view = factory.view
view.table

,id,0
0,1,test_1
1,2,test_2
2,3,test_3
3,4,test_4
4,0,test


In [8]:
db.create("test")


🆕 새로운 ID 생성: 4


AttributeError: 'str' object has no attribute 'as_posix'

In [7]:
work = db.get(2)

In [8]:
work.task_db

***violating paths***


TaskDB(dir_db=DirDB(dir_path=WindowsPath('W:/RM/resources/works/test_2___id_2'), factory=<rm.dirtree.factory.DirTreeFactory object at 0x000001B8C21F8940>), factory=TaskResourceFactory(dir_path=WindowsPath('W:/RM/resources/works/test_2___id_2'), CONFIG_MANAGER_CLASS=<class 'mmdetection_rm.work.task.TaskConfigManager'>, RECORD_CLASS=<class 'mmdetection_rm.work.task.TaskRecord'>, DB_CLASS=<class 'mmdetection_rm.work.task.TaskDB'>, VIEW_CLASS=<class 'mmdetection_rm.work.task.TaskDBView'>, CONFIG_NAME='task_config'), RECORD_CLASS=<class 'mmdetection_rm.work.task.TaskRecord'>)

In [10]:
work.create_train_task(1, 1, 1)

🆕 새로운 ID 생성: 0


TaskRecord(id=0, name='train', dir_path=WindowsPath('W:/RM/resources/works/test_2___id_2/train___id_0'), config_manager=TaskConfigManager(dir_path=WindowsPath('W:/RM/resources/works/test_2___id_2/train___id_0'), memo_factory=<rm.memo.factory.MemoFactory object at 0x000001B8E24E6590>, CONFIG_NAME='task_config'))

In [11]:
work.task_view.table

,id,0
0,0,train


In [2]:
from resource_manager_mmdetection.env_config import project_root

project_root

PosixPath('/home/submodules/mmdetection')